In [1]:
import json, gzip

path = "train.jsonl.gz"
data = []
i = 0
with gzip.open(path) as f:
    for ln in f:
        i+=1
        if i>5000:
            break
        obj = json.loads(ln.decode('utf-8'))
        data.append(obj)
        
            

In [2]:
print(len(data))

5000


In [3]:
# a = "He's a good boy. He didn't go? Tribune will track the world\'s most popular sporting event."
# a = re.sub(r"([.!?,'])", r" \1", a.lower())
# a = re.split('\*|\n| ',a)
# print(a)
# # print(data[0]['summary'])

In [4]:
# print(data[0])


In [5]:
# print(type(data[0]))

In [6]:
# print(type(data[2]['text']))

In [7]:
import torch
import torch.nn as nn

In [8]:
data1 = []
for i in range(len(data)):
    data1.append( {k: data[i][k] for k in ('summary', 'text')})

In [9]:
print(data1[0].keys())
print(type(data1[0]))

dict_keys(['text', 'summary'])
<class 'dict'>


In [10]:
#Vocab creation
import re
import unicodedata
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
UNK_token = 3  # Unknown word token
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {"PAD":PAD_token, "SOS":SOS_token, "EOS":EOS_token, "UNK":UNK_token}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token:"UNK"}
        self.num_words = 4  # Count SOS, EOS, PAD, UNK
        
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

In [23]:
MAX_LENGTH = 200  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def tokenizeNnormalize(a):
    a = unicodeToAscii(a.lower().strip())
    a = re.sub(r"([.!?,'])", r" \1", a.lower())
    a = re.split('\*|\n| ',a)
    return a

def tokenToIndex(tokens):
    a = []
    for t in tokens:
        if t in voc.word2index:
            a.append(voc.word2index[t])
        else:
            a.append(voc.word2index['UNK'])
    return a    

In [24]:
# tokenizeNnormalize("he-will's.")

In [25]:
import nltk.data
from nltk import tokenize
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
voc = Voc("tmp")
for d in data1:
    temp = d['text']
    temp = tokenizeNnormalize(temp)
    for word in temp:
            voc.addWord(word)
    temp = d['summary']
    temp = tokenizeNnormalize(temp)
    for word in temp:
            voc.addWord(word)

In [26]:
print(len(voc.word2index))

126405


In [52]:
#Train set
import json, gzip
path = "train.jsonl.gz"
data = {}
train_data = []
i = 0
with gzip.open(path) as f:
    for ln in f:
        i+=1
        if i>5000:
            break
        obj = json.loads(ln.decode('utf-8'))
        a = tokenizeNnormalize(obj['text'])
        if(len(a)>298):
            continue
        
        #padding
        a = ['SOS'] + a + ['EOS']
        while(len(a)<300):    
            a.append('PAD')   
        
        t = {}    
        t['text'] = tokenToIndex(a)
        
        a = tokenizeNnormalize(obj['summary'])
        a = ['SOS'] + a + ['EOS'] 
        
        while(len(a)<170):    
            a.append('PAD')
        
        t['summary'] = tokenToIndex(a)
        train_data.append(t)
data['train'] =  train_data       

In [53]:
#Dev set

path = "dev.jsonl.gz"
dev_data = []
i = 0
with gzip.open(path) as f:
    for ln in f:
        i+=1
        if i>1000:
            break
        obj = json.loads(ln.decode('utf-8'))
        a = tokenizeNnormalize(obj['text'])
        if(len(a)>298):
            continue
        a = ['SOS'] + a + ['EOS']    
         #padding
        while(len(a)<300):    
            a.append('PAD')
            
        t = {}    
        t['text'] = tokenToIndex(a)
        a = tokenizeNnormalize(obj['summary'])
        a = ['SOS'] + a + ['EOS']  
        
        while(len(a)<170):    
            a.append('PAD')
        
        t['summary'] = tokenToIndex(a)
        dev_data.append(t)
data['dev'] =  dev_data       

In [54]:
#Test Data

path = "test.jsonl.gz"
test_data = []
i = 0
with gzip.open(path) as f:
    for ln in f:
        i+=1
        if i>1000:
            break
        obj = json.loads(ln.decode('utf-8'))
        a = tokenizeNnormalize(obj['text'])
        if(len(a)>298):
            continue
         #padding
        a = ['SOS'] + a + ['EOS'] 
        while(len(a)<300):    
            a.append('PAD')
           
        t = {}    
        t['text'] = tokenToIndex(a)
        a = tokenizeNnormalize(obj['summary'])
        a = ['SOS'] + a + ['EOS']  
        
        while(len(a)<170):    
            a.append('PAD')
        
        t['summary'] = tokenToIndex(a)
        test_data.append(t)
data['test'] =  test_data       

In [62]:
import numpy as np
batch = train_data[0:1]
batch_text = np.zeros([len(batch), len(batch[0]['text'])])
batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
i = 0
for d in batch:
    batch_text[i] = d['text']
    batch_summary[i] = d['summary']
    i += 1


In [61]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def compute_perplexity(dataset, net, bsz=64):
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
#     num_examples, seq_len = dataset.size()
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in\
               range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        i = 0
        for d in batch:
            batch_text[i] = d['text']
            batch_summary[i] = d['summary']
            i += 1

        ut = torch.nonzero(batch_text).size(0)
        preds = net(batch_text, batch_summary)
        targets = batch_summary[:, 1:].contiguous().view(-1)
        preds = preds[:, :-1, :].contiguous().view(-1, net.vocab_size)
        loss = criterion(preds, targets)
        nll += loss.detach()
        total_unmasked_tokens += ut

    perplexity = torch.exp(nll / total_unmasked_tokens).cpu()
    return perplexity.data


def train_lm(dataset, params, net):
    
    # since the first index corresponds to the PAD token, we just ignore it
    # when computing the loss
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = len(dataset)    
    batches = [(start, start + params['batch_size']) for start in\
               range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        random.shuffle(batches)
        
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            
            batch_text = np.zeros([len(batch), len(batch[0]['text'])])
            batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
            i = 0
            for d in batch:
                batch_text[i] = d['text']
                batch_summary[i] = d['summary']
                i += 1

            preds = net(batch_text, batch_summary)
            
            preds = preds[:, :-1, :].contiguous().view(-1, net.vocab_size)
            targets = batch_summary[:, 1:].contiguous().view(-1)
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss

        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, compute_perplexity(data['dev'], net)))

In [64]:
class SelfAttention(nn.Module):
    def __init__(self, params):
        super(SelfAttention, self).__init__()
        self.vocab_size = params['vocab_size']
        self.d_emb = params['d_emb']
        self.embeddings = nn.Embedding(self.vocab_size, self.d_emb)
        
    def forward(self, batch):
        pass    

In [65]:
params = {}
params['vocab_size'] = len(voc.word2index)
params['d_emb'] = 512
params['d_hid'] = 512
params['batch_size'] = 50
params['epochs'] = 5
params['learning_rate'] = 0.001

net = SelfAttention(params)

In [81]:
###Universal Sentence Embedding

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
# from sklearn.metrics.pairwise import cosine_similarity

W0404 18:21:31.275345 10596 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [8]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

Instructions for updating:
Colocations handled automatically by placer.


W0331 22:47:02.741527 11132 deprecation.py:323] From C:\Users\Shruti1992\Anaconda3\envs\cs682\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
import nltk.data
from nltk import tokenize
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data1 = data[2]['text']
# print ('\n-----\n'.join(tokenizer.tokenize(data1)))
sentences_list = tokenize.sent_tokenize(data1)

In [10]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed(sentences_list))

In [11]:
print(sentences_embeddings)

[[ 0.07024275  0.04368595 -0.03332052 ...  0.03477242 -0.0144234
  -0.01740775]
 [ 0.05018305  0.04429227 -0.03065547 ... -0.03877924 -0.00763866
   0.04358229]
 [ 0.05427796  0.05102638 -0.02369512 ... -0.03321327  0.02231499
  -0.02626558]
 ...
 [ 0.06598777  0.02679331 -0.05378729 ... -0.06943782 -0.02672804
  -0.0065122 ]
 [ 0.01991204  0.03930764 -0.03562501 ... -0.01346327 -0.01935354
   0.02114006]
 [ 0.02234399 -0.04831612 -0.03079941 ... -0.06042011 -0.07132834
  -0.03533927]]
